# Deploy trained Keras or TensorFlow models using Amazon SageMaker
https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

## Step 1. Set up

In [ ]:
!pip uninstall keras -y

In [ ]:
!pip install keras==2.2.4

In [ ]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

## Step 2. Upload Weights and JSON model files manually

In [ ]:
!mkdir keras_model

In [ ]:
%%sh

aws s3 cp s3://abaykov/model_architecture.json ./keras_model/
aws s3 cp s3://abaykov/model_weights.h5 ./keras_model/

In [ ]:
!ls -l keras_model

## Step 3. Load the Keras model using the JSON and weights file

In [ ]:
import keras
from keras.models import model_from_json

json_file = open('/home/ec2-user/SageMaker/keras_model/'+'model_architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/model_weights.h5')

print("Loaded model from disk")

## Step 4. Export the Keras model to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

## Step 5. Convert TensorFlow model to an Amazon SageMaker-readable format

In [ ]:
!ls export

!ls export/Servo

!ls export/Servo/1

!ls export/Servo/1/variables

import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
    
import sagemaker
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

## Step 6. Deploy the trained model

In [ ]:
!touch train.py

endpoint_name = 'ayrat-binary-classification'

from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

# %%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge',
                                   endpoint_name=endpoint_name)

In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

## Step 7. Invoke the endpoint

In [ ]:
!pip install opencv-python

In [ ]:
import numpy as np
import cv2

In [ ]:
%%sh
aws s3 cp s3://abaykov/test.jpg /tmp/test.jpg

In [ ]:
file_name = '/tmp/test.jpg'
from IPython.display import Image
Image(file_name)

In [ ]:
def loadImage(filename):
    img = cv2.imread(filename, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224, 150))
    img = img / 255
    # Reshape from (150,224) to (1,150,224,3) : 1 sample, 150x224 pixels, 3 channels
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=0)
    img = np.reshape(img, (1,150,224,3))
    return np.array(img)
wm_image_array = loadImage(file_name)
result = predictor.predict(wm_image_array)
print('The class is predicted as', format(result.get('outputs').get('score').get('float_val')[0],'.16f'), 'between 0.00 (class 1) and 1.00 (class 2)')

## Step 8. Clean up (work on it while running)

In [ ]:
print(endpoint_name)
print(sagemaker_model)
sagemaker_session.delete_endpoint(endpoint_name)
#TODO sagemaker_session.delete_endpoint_config(EndpointConfigName=endpoint_name)
#TODO sagemaker_session.delete_model(ModelName=endpoint_name)
# I did not have time to battle deletion of all objects in AWS, so I have TODOs above. For the interest of time
# I just deleted the rest of the things manually.